In [2]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/Documents/git_workspace/16715/dynamics-simulation-leg/scripts/Project.toml`


In [3]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays
using ForwardDiff

WebIO._IJuliaInit()

In [4]:
# world = RigidBody{Float64}("world")
# doublependulum = Mechanism(world; gravity = SVector(0, 0, g))
curdir = pwd()
urdfpath = joinpath(curdir, "../res/flyhopper_robot/urdf/flyhopper_robot.urdf")
doublependulum = parse_urdf(urdfpath, floating=false)
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [5]:
const N = 4 # of links

# link lengths
const l0 = 0.1
const l1 = 0.3
const l2 = 0.3
const l3 = 0.1
const l4 = 0.2
const l5 = 0.0205
const lee = [l3 + l4; l5; 0] # sqrt((l3 + l4)^2 + l5^2)

# CoM locations
const l_cb = [0; 0.004; 0]
const l_c0 = [0.0125108364230515; 0.00117191218927888; 0]
const l_c1 = [0.149359714867044; 0; 0]
const l_c2 = [0.0469412900551914; 0; 0]
const l_c3 = [0.113177000131857; -0.015332867880069; 0]

# link masses
const mb = 7  # kg
const m0 = 0.24644240965487
const m1 = 0.0707939028219395
const m2 = 0.276735496985514
const m3 = 0.130824780046739
const m = diagm([m0, m1, m2, m3])
    
# gravity, obviously
const g = 9.807

# mass moment of inertia in axis of rotation
const Ib = Array([0.0024241 5.252E-06 2.0733E-19; 
                  5.252E-06 0.0044176 -3.1153E-19; 
                  2.0733E-19 -3.1153E-19 0.0022481])

const I0 = Array([3.83120149546952E-05 1.46925714738609E-05 -8.60106401672571E-06;
                  1.46925714738609E-05 0.000172067745507247 1.0427260925207E-06;
                  -8.60106401672571E-06 1.0427260925207E-06 0.00014745218068435])

const I1 = Array([3.06999775886187E-06 7.91090301514898E-12 -1.43705963146176E-12;
                  7.91090301514898E-12 0.000147960574744097 1.30742394049546E-11;
                  -1.43705963146176E-12 1.30742394049546E-11 0.000147884231885009])

const I2 = Array([3.43038397803592E-05 -2.90339844227483E-07 6.18680397558952E-06;
                  -2.90339844227483E-07 0.000302324068012293 2.25016327583562E-08;
                  6.18680397558952E-06 2.25016327583562E-08 0.00028292376778719])

const I3 = Array([1.76996970020568E-05 -5.3695427116208E-07 7.62350214406387E-07;
                  -5.3695427116208E-07 0.000164188445564489 -2.77843753828047E-07;
                  7.62350214406387E-07 -2.77843753828047E-07 0.000160656046697151])
# const J = Diagonal([I0, I1, I2, I3])

3×3 Matrix{Float64}:
  1.76997e-5  -5.36954e-7    7.6235e-7
 -5.36954e-7   0.000164188  -2.77844e-7
  7.6235e-7   -2.77844e-7    0.000160656

In [6]:
M̄ = [mb*I(3) zeros(3, 27)
     zeros(3,3) Ib zeros(3, 24)
     zeros(3,6) m0*I(3) zeros(3,21);
     zeros(3,9) I0 zeros(3,18);
     zeros(3,12) m1*I(3) zeros(3,15);
     zeros(3,15) I1 zeros(3, 12);
     zeros(3,18) m2*I(3) zeros(3, 9);
     zeros(3, 21) I2 zeros(3, 6);
     zeros(3, 24) m3*I(3) zeros(3, 3);
     zeros(3, 27) I3]

30×30 SparseArrays.SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⠑⢄⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⠛⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⢱⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠑⢄⣀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢄⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢱⣶⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛

In [7]:
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

H = [zeros(1,3); I];

T = Diagonal([1.0; -1; -1; -1])

function G(Q)
    return L(Q)*H
end

function Ḡ(q)
    Q = q[4:7]
    return [I(3) zeros(3,4); zeros(4,3) G(Q) zeros(4,1); zeros(1,6) 1]
end

function G̃(q)
    Q = q[4:7]
    #return [I(3) zeros(3,4); zeros(4,3) G(Q) zeros(4,1); zeros(1,6) 1]
    return [I(3) zeros(3,4); zeros(4,3) 0.5*G(Q) zeros(4,1); zeros(1,6) 1]
end

G̃ (generic function with 1 method)

In [8]:
function Expq(ϕ)
    # The quaternion exponential map ϕ → Q 
    # Convert axis angle to quaternion
    Q = zeros(eltype(ϕ), 4)
    θ = norm(ϕ)
    r = ϕ/θ
    Q[1] = cos(θ/2)
    Q[2:4] = r*sin.(θ/2)
    return Q
end

function rotate(Q, p)
    # Rotate a position vector p by a quaternion Q
    return H'L(Q)*R(Q)'*H*p
end

rotate (generic function with 1 method)

In [ ]:
#Kinematics
function kinematics(q)
    rb = q[1:3]  # body com location
    Qb = q[4:7]  # body quaternion
    q0 = q[8]  # joint angles
    q1 = q[9]
    q2 = q[10]
    q3 = q[11]
    
    pb = rb + rotate(Qb, l_cb)  # position vector from world frame to *JOINTS* 0 and 2
    
    Qb0 = Expq([0, 0, q0])  # quaternion from base to link 0
    Q0 = L(Qb)*Qb0  # quaternion from world frame to link 0
    r0 = pb + rotate(Q0, l_c0)
    
    Q01 = Expq([0, q1, 0])  # quaternion from link 0 to link 1
    Q1 = L(Q0)*Q01  # quaternion from world frame to link 1
    r1 = pb + rotate(Q0, [l0; 0; 0]) + rotate(Q1, l_c1)  
    
    Qb2 = Expq([0, q2, 0])  # quaternion from base to link 2
    Q2 = L(Qb)*Qb2  # quaternion from world frame to link 2
    r2 = pb + rotate(Q2, l_c2)
    
    Q23 = Expq([0, q3, 0])  # quaternion from base to link 2
    Q3 = L(Q2)*Q23  # quaternion from world frame to link 2
    r3 = pb + rotate(Q2, [l2; 0; 0]) + rotate(Q3, l_c3)  
    
    return [rb; Qb; r0; Q0; r1; Q1; r2; Q2; r3; Q3]
end

In [ ]:
#Kinematic Jacobian
function K(q)
    k = kinematics(q);
    rb = k[1:3]
    Qb = k[4:7]
    r0 = k[8:10]
    Q0 = k[11:14]
    r1 = k[15:17]
    Q1 = k[18:21]
    r2 = k[22:24]
    Q2 = k[25:28]
    r3 = k[29:31]
    Q3 = k[32:35]
    
    dk = ForwardDiff.jacobian(dq->kinematics(dq),q)
    @show size(dk)
    K = [I(3) zeros(3,11);
        zeros(3,3) 2*G(Q0)' zeros(3,7);
        zeros(3,7) I(3) zeros(3,4);
        zeros(3,10) 2*G(Q1)']
        *dk*
        [I(3) zeros(3,4);
        zeros(4,3) 0.5*G(Q0) zeros(4,1);
        zeros(1,6) 1.0]
    return K
end

In [ ]:
#Mass Matrix
function M(q)    
    return K(q)'*M̄*K(q)
end

# eigvals(M(q0))

In [ ]:
#Potential
function U(q)
    return 0.0  # assume zero gravity for now
end

function DQ0U(q)
    rb = q[1:3]  # body com location
    Qb = q[4:7]  # body quaternion
    q0 = q[8]  # joint angles
    q1 = q[9]
    q2 = q[10]
    q3 = q[11]
    
    ForwardDiff.gradient(dQ0->U([r0; dQ0; q1]), Q0) 
end

In [ ]:
#Lagrangian
function Lagrangian(q,v)
    0.5*v'*M(q)*v - U(q)
end

In [ ]:
#Lagrangian Derivatives

function Dr0L(q,v)
    rb = q[1:3]  # body com location
    Qb = q[4:7]  # body quaternion
    q0 = q[8]  # joint angles
    q1 = q[9]
    q2 = q[10]
    q3 = q[11]
    
    ForwardDiff.gradient(dr0->Lagrangian([dr0; Q0; q1],v), r0)
end

function DQ0L(q,v)
    rb = q[1:3]  # body com location
    Qb = q[4:7]  # body quaternion
    q0 = q[8]  # joint angles
    q1 = q[9]
    q2 = q[10]
    q3 = q[11]
    
    ForwardDiff.gradient(dQ0->Lagrangian([r0; dQ0; q1],v), Q0)
end

function Dq1L(q,v)
    rb = q[1:3]  # body com location
    Qb = q[4:7]  # body quaternion
    q0 = q[8]  # joint angles
    q1 = q[9]
    q2 = q[10]
    q3 = q[11]
    
    ForwardDiff.derivative(dq1->Lagrangian([r0; Q0; dq1],v), q1)
end

function Dṙ0L(q,v)
    ṙb = v[1:3]
    ωb = v[4:6]
    q̇0 = v[7]
    q̇1 = v[8]
    q̇2 = v[9]
    q̇3 = v[10]
    
    ForwardDiff.gradient(dṙ0->Lagrangian(q,[dṙ0;ω0;q̇1]), ṙ0)
end
    
function Dω0L(q,v)
    ṙb = v[1:3]
    ωb = v[4:6]
    q̇0 = v[7]
    q̇1 = v[8]
    q̇2 = v[9]
    q̇3 = v[10]
    
    ForwardDiff.gradient(dω0->Lagrangian(q,[ṙ0;dω0;q̇1]), ω0)
end

function Dq̇1L(q,v)
    ṙb = v[1:3]
    ωb = v[4:6]
    q̇0 = v[7]
    q̇1 = v[8]
    q̇2 = v[9]
    q̇3 = v[10]
    
    ForwardDiff.derivative(dq̇1->Lagrangian(q,[ṙ0;ω0;dq̇1]), q̇1)
end
    

In [ ]:
#Input Jacobian
function B(q)
    #[I(3) zeros(3); zeros(3,4); zeros(1,3) 1.0]
    [zeros(6); 1.0]
end

In [ ]:
#Initial Conditions
rb = zeros(3)
Qb = [1.0; 0; 0; 0]
q0 = 0.0
q1 = 0.0
q2 = 0.0
q3 = 0.0
q_init = [rb; Qb; q0; q1; q2; q3]

q̇0 = 0.0
q̇1 = 0.0
q̇2 = 0.0
q̇3 = 0.0

ωb = zeros(3)
ṙb = zeros(3) #cross(ω0,[-0.5*ℓ1; 0; 0])

v_init = [ṙb; ωb; q̇0; q̇1; q̇2; q̇3]
x_init = [q_init; v_init]

In [ ]:
#EL Dynamics
function f(x,u)
    q = x[1:11]
    v = x[12:21]
    ω = v[4:6]
    
    q̇ = G̃(q)*v
    
    Mq = [ForwardDiff.jacobian(dv->Dṙ0L(q,dv),v);
         ForwardDiff.jacobian(dv->Dω0L(q,dv),v);
         ForwardDiff.gradient(dv->Dq̇1L(q,dv),v)']
    
    v̇ = Mq\([-ForwardDiff.jacobian(dq->Dṙ0L(dq,v),q)*G̃(q)*v + Dr0L(q,v);
             -ForwardDiff.jacobian(dq->Dω0L(dq,v),q)*G̃(q)*v - hat(ω)*Dω0L(q,v) + 0.5*G(Q0)'*DQ0U(q);
             -ForwardDiff.gradient(dq->Dq̇1L(dq,v),q)'*G̃(q)*v + Dq1L(q,v)] + B(q)*u)
    
    #ForwardDiff.jacobian(dv->Dṙ0L(q,dv),v)*v̇ + ForwardDiff.jacobian(dq->Dṙ0L(dq,v),q)*G̃(q)*v - Dr0L(q,v)
    #ForwardDiff.jacobian(dv->Dω0L(q,dv),v)*v̇ + ForwardDiff.jacobian(dq->Dω0L(dq,v),q)*G̃(q)*v + hat(ω)*Dω0L(q,v) - 0.5*H'*L(Q0)'*DQ0L(q,v)
    #ForwardDiff.jacobian(dv->Dq̇1L(q,dv),v)*v̇ + ForwardDiff.jacobian(dq->Dq̇1L(dq,v),q)*G̃(q)*v - Dq1L(q,v)
    
    
    return [q̇; v̇]
end

In [ ]:
function constraint(q)    
    
    x0 = l0*cos(q[1])
    y0 = l0*sin(q[1])
    x1 = x0 + l1*cos(q[1] + q[2])
    y1 = y0 + l1*sin(q[1] + q[2])
    
    # constraint forward kinematics
    xc = l2*cos(q[3]) + l3*cos(q[3] + q[4])
    yc = l2*sin(q[3]) + l3*sin(q[3] + q[4])
    
    c = zeros(eltype(q), 2)
    c[1] = x1 - xc
    c[2] = y1 - yc
    
    return c
end

In [ ]:
function jacobian_constraint(q)
    # D = zeros(eltype(q), 2)
    D = ForwardDiff.jacobian(_q -> constraint(_q), q)
    return D
end

In [ ]:
function dqdot(q, q̇)
    D = jacobian_constraint(q)
    dqdot = D * q̇
    return dqdot
end

In [ ]:
function derivative_constraint_jac(q, q̇)
    # Compute del/delq(D(q)q_dot)q_dot
    d = ForwardDiff.jacobian(_q -> dqdot(_q, q̇), q) * q̇
    return d
end

In [ ]:
function cdot(q, q̇)
    #Compute cdot (first derivative of constraint function)
    D = jacobian_constraint(q)
    cdot = q̇' * D'
    return cdot
end

In [ ]:
function f(x)
    # parallel mechanism dynamics
    
    q = x[1:N]
    q̇ = x[N+1:end]
    
    # Constraint fn
    c = constraint(q)
    
    # Constraint Jacobian
    D = jacobian_constraint(q)
    
    # del/delq(D*qdot)*qdot
    d = derivative_constraint_jac(q, q̇)
    
    # first derivative of constraint fn
    cd = cdot(q, q̇)'
    
    ẋ = zeros(eltype(x),length(x))
    
    # Mass matrix
    M = Mq(q)
    
    # Coriolis matrix
    C = Cq(q, q̇)
    
    # Potential matrix
    G = Gq(q)
    
    α = 10.0
    β = 1
    
    e = D*(M\(D'*(α*c + β*cd)))
    
    KKT = [M -D';
           D zeros(2,2)]
    RHS = [- G - C;
           -(d + e)]
    
    sol = KKT\RHS
    λ = sol[5:6]
    
    ẋ[1:N] = q̇
    ẋ[N+1:end] = sol[1:4]  # q double dot
    
    return ẋ
end

In [ ]:
function checkval(val, max, name)
    if val > max
        @show val
        error(name)
    end
end
#=
#---#
if isnan(qdd[1])
    @show qdd
    error("A!")
end
#---#
=#

In [ ]:
function rk4_step(f,xk,h)

    f1 = f(xk)
    f2 = f(xk + 0.5*h*f1)
    f3 = f(xk + 0.5*h*f2)
    f4 = f(xk + h*f3)
    
    xn = xk + (h/6.0).*(f1 + 2*f2 + 2*f3 + f4)

    return xn
end

In [ ]:
Tf = 10.0
h = 0.001 #20 Hz
n = Int(floor(Tf./h + 1))
thist = h.*Array(0:(n-1));

In [ ]:
function simulate!(xtraj, n)
    for k = 1:(n-1)
        xtraj[:,k+1] .= rk4_step(f, copy(xtraj[:,k]), h)
    end
end

In [ ]:
x0 = [-30*pi/180; -120*(pi/180); -150*(pi/180); 120*(pi/180); 0.0; 0.0; 0.0; 0.0]
xtraj = zeros(8,n)
xtraj[:,1] = x0;

In [ ]:
simulate!(xtraj, n)

In [ ]:
q0 = -xtraj[1, :] .- 30*(pi/180)
q1 = -xtraj[2, :] .- 120*(pi/180)
q2 = -xtraj[3, :] .- 150*(pi/180)
q3 = -xtraj[4, :] .+ 120*(pi/180)

qs = convert(AbstractArray{Float64}, [q0 q2 q1 q3]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 

In [ ]:
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(doublependulum, URDFVisuals(urdfpath));

render(mvis)

In [ ]:
# [q0 q2 q1 q3]
set_configuration!(mvis, [-x0[3]-150*(pi/180), -x0[1]-30*(pi/180), -x0[4]+120*(pi/180), -x0[2]-120*(pi/180)])

In [ ]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

#animation = Animation(mvis, ts, -q_array)
#setanimation!(mvis, animation)

MeshCatMechanisms.animate(mvis, ts, -q_array; realtimerate = 1.);

In [ ]:
#Energy Functions

function H(x)
    q = x[1:N]
    q̇ = x[N+1:end]
    energy = Lagrangian(q,q̇)
    return energy
end

In [ ]:
#Plot total energy
using Plots

E = zeros(n)
for k = 1:n
    E[k] = H(xtraj[:, k])
end

plot(thist,E)